In [1]:
import pandas as pd
import pandas as pd
import numpy as np

In [2]:
ascend_att_all =  pd.read_csv('data/attendance_ascend_full.csv')
ascend_dashboard = pd.read_excel('data/ascend_attendance_dashboard.xls')
ascend_info_active = pd.read_csv('data/ascend_student_info_active.csv')
ascend_info_former = pd.read_csv('data/ascend_student_info_former.csv')

## 1. Clean Attendance Files

In [ ]:
ascend_attendance_active.dropna(subset=['Student.1'], inplace = True)

# Chose subset of columns to keep
ascend_attendance_active = ascend_attendance_active[['Student', 'Grade', 'Homeroom', 'Member', 'Present', 'Absent', 'Tardy', 'Dismiss']]

# filter out renments of excel pagination
ascend_attendance_active = ascend_attendance_active.loc[ascend_attendance_active['Student'] != "CHARTER"]
ascend_attendance_active = ascend_attendance_active.loc[ascend_attendance_active['Student'] != "Student"]

In [ ]:
def cleanAttendance(attendance_file):
    
    # Drop empty rows
    attendance_file.dropna(subset=['Student.1'], inplace = True)

    # Chose subset of columns to keep
    attendance_file = attendance_file[['Student', 'Grade', 'Homeroom', 'Member', 'Present', 'Absent', 'Tardy', 'Dismiss']]

    # filter out renments of excel pagination
    attendance_file = attendance_file.loc[attendance_file['Student'] != "CHARTER"]
    attendance_file = attendance_file.loc[attendance_file['Student'] != "Student"]
    attendance_file = attendance_file.loc[~attendance_file['Student'].str.contains("Page")]
    
    return(attendance_file)

In [ ]:
ascend_active_attendance_clean = cleanAttendance(ascend_active_attendance)
ascend_former_attendance_clean = cleanAttendance(ascend_former_attendance)

In [ ]:
ascend_active_attendance_clean.to_csv('data/ascend_active_attendance_final.csv')

In [ ]:
ascend_former_attendance_clean.to_csv('data/ascend_former_attendance_final.csv')

## 2. Why Are there additional students in the file from ASPEN?

### 2A. Remove Middle Initial from Dashboard file

In [ ]:
# Importing re package for using regular expressions 
import re 
  
# Function to clean the names 
def Clean_names(student_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if len(student_name.split(",")) == 3:
        return re.search('(.*)\,', student_name).group(1)
    else:
        return student_name
    
def clean_names2(student_name):
    line = re.sub('Jr', '', student_name)
    
    return line


In [ ]:
ascend_dashboard['Student Name'] = ascend_dashboard['Student Name'].apply(Clean_names)

In [ ]:
ascend_attendance = ascend_active.append(ascemd_former, ignore_index=True)
ascend_attendance_deduped = ascend_attendance.drop_duplicates()

In [ ]:
dashboard = ascend_dashboard[['Student Name']]
dashboard.rename(columns = {'Student Name': 'Student'}, 
                       inplace=True)
aspen = ascend_attendance_deduped[['Student']]

In [ ]:
merged = pd.merge(aspen, 
         dashboard, 
         on = ['Student'],
         how='outer',
         indicator=True)

In [ ]:
merged['which_df'] = np.where(
     merged['_merge'] == "left_only", 
    'aspen', 
     np.where(
        merged['_merge'] == "right_only", 'dashboard', 'both'
     )
)

In [ ]:
merged.sort_values(['which_df'], inplace = True)
merged.reset_index(inplace=True)

In [ ]:
merged.drop(['index'], axis=1, inplace=True)

In [ ]:
merged = merged.loc[merged['which_df'] != 'both']

## 3. Add Student Numbers to Student Attendance

In [3]:
ascend_info = ascend_info_active.append(ascend_info_former, ignore_index=True)

In [4]:
ascend_att_all.drop_duplicates(inplace=True)
ascend_info.drop_duplicates(inplace=True)

In [5]:
ascend_info.columns

Index(['School > Name', 'Name', 'Student ID', 'Grade', 'Homeroom',
       'EnrStatus'],
      dtype='object')

In [6]:
ascend_info = ascend_info[['Name', 'Student ID', 'Grade', 'Homeroom', 'EnrStatus', 'School > Name']]

In [7]:
aspen_attendance_joined = pd.merge(ascend_att_all, 
                                   ascend_info, 
                                   left_on=['Student','Homeroom', 'Grade'], 
                                   right_on=['Name','Homeroom', 'Grade'], 
                                   how='left'
                                  )

In [8]:
aspen_attendance_joined = aspen_attendance_joined.loc[aspen_attendance_joined['Member'] != 0]

In [9]:
aspen_attendance_joined

,Student,Grade,Homeroom,Member,Present,Absent,Tardy,Dismiss,Name,Student ID,EnrStatus,School > Name
4,"Adams, Amarion",K,114,121,120.0,1.0,0,0,"Adams, Amarion",50751787.0,Enrolled,KIPP - ASCEND
7,"Adams, Jeremiah",04,301P,121,120.0,1.0,0,0,"Adams, Jeremiah",50363643.0,Enrolled,KIPP - ASCEND
8,"Adams, Mercutio",06,310,120,120.0,0.0,0,0,"Adams, Mercutio",50281845.0,Enrolled,KIPP - ASCEND
10,"Adams, Natalie",02,205,56,55.0,1.0,0,0,"Adams, Natalie",50555129.0,NotEnrolled,KIPP - ASCEND
11,"Adelaja, Olufunmisho",K,114,121,112.0,9.0,0,0,"Adelaja, Olufunmisho",50620222.0,Enrolled,KIPP - ASCEND
...,...,...,...,...,...,...,...,...,...,...,...,...
1378,"Wright, William",04,A207,95,81.0,14.0,0,0,"Wright, William",50320329.0,Enrolled,KIPP - ASCEND
1379,"Wyatt, Demere",04,303P,121,119.0,2.0,0,0,"Wyatt, Demere",50461313.0,Enrolled,KIPP - ASCEND
1380,"Yancy, Dashyia",03,307P,121,118.0,3.0,0,0,"Yancy, Dashyia",50483221.0,Enrolled,KIPP - ASCEND
1381,"Yelvington, Tristen",02,203,117,111.0,6.0,0,0,"Yelvington, Tristen",50715294.0,Enrolled,KIPP - ASCEND


In [10]:
ascend_attendance = aspen_attendance_joined[['Student', 'Student ID', 'Grade', 'Homeroom', 'EnrStatus', 
                         'School > Name', 'Member', 'Present', 'Absent']]

In [11]:
ascend_attendance['Attendance Pct'] = (ascend_attendance['Present']/ ascend_attendance['Member'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
ascend_dashboard_filtered = ascend_dashboard.loc[ascend_dashboard['Membership Days'] != 1]

In [13]:
ascend_dashboard_filtered

,Student Name,Student ID,Current Grade Level,Current Homeroom,Current School,Attendance School,Attendance Network,Attendance Pct,Timeframe,Membership Days,Absences
21,"Dickey, HANNA, M.",50588591,K,--,*INACTIVE*,KIPP - ASCEND,Charter,1.000000,2019-2020,2,0.0
22,"Dickey, Isabella, M.",50611658,2,--,*INACTIVE*,KIPP - ASCEND,Charter,1.000000,2019-2020,2,0.0
23,"Gardner, Ashanti, M.",50242697,7,--,*INACTIVE*,KIPP - ASCEND,Charter,1.000000,2019-2020,2,0.0
24,"Hill, Deniah, L.",50257780,4,--,*INACTIVE*,KIPP - ASCEND,Charter,1.000000,2019-2020,2,0.0
25,"Howard, Jason, A. Jr",50114155,7,--,*INACTIVE*,KIPP - ASCEND,Charter,1.000000,2019-2020,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...
984,"WRIGHT, SHAELA, C.",50582800,1,217B,KIPP - ASCEND,KIPP - ASCEND,Charter,0.965517,2019-2020,116,4.0
985,"Wright, Treyon",50247181,8,311,KIPP - ASCEND,KIPP - ASCEND,Charter,0.991379,2019-2020,116,1.0
986,"Wyatt, Demere",50461313,4,303P,KIPP - ASCEND,KIPP - ASCEND,Charter,0.982758,2019-2020,116,2.0
987,"Yancy, Dashyia, H.",50483221,3,307P,KIPP - ASCEND,KIPP - ASCEND,Charter,0.974137,2019-2020,116,3.0


In [14]:
ascend_attendance['Timeframe'] = '2019-2020'

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
ascend_attendance

,Student,Student ID,Grade,Homeroom,EnrStatus,School > Name,Member,Present,Absent,Attendance Pct,Timeframe
4,"Adams, Amarion",50751787.0,K,114,Enrolled,KIPP - ASCEND,121,120.0,1.0,0.991736,2019-2020
7,"Adams, Jeremiah",50363643.0,04,301P,Enrolled,KIPP - ASCEND,121,120.0,1.0,0.991736,2019-2020
8,"Adams, Mercutio",50281845.0,06,310,Enrolled,KIPP - ASCEND,120,120.0,0.0,1.000000,2019-2020
10,"Adams, Natalie",50555129.0,02,205,NotEnrolled,KIPP - ASCEND,56,55.0,1.0,0.982143,2019-2020
11,"Adelaja, Olufunmisho",50620222.0,K,114,Enrolled,KIPP - ASCEND,121,112.0,9.0,0.925620,2019-2020
...,...,...,...,...,...,...,...,...,...,...,...
1378,"Wright, William",50320329.0,04,A207,Enrolled,KIPP - ASCEND,95,81.0,14.0,0.852632,2019-2020
1379,"Wyatt, Demere",50461313.0,04,303P,Enrolled,KIPP - ASCEND,121,119.0,2.0,0.983471,2019-2020
1380,"Yancy, Dashyia",50483221.0,03,307P,Enrolled,KIPP - ASCEND,121,118.0,3.0,0.975207,2019-2020
1381,"Yelvington, Tristen",50715294.0,02,203,Enrolled,KIPP - ASCEND,117,111.0,6.0,0.948718,2019-2020


In [47]:
no_matches = pd.merge(ascend_dashboard_filtered, ascend_attendance, 
       on = ['Student ID'], 
       how='outer',
       indicator=True
      )

no_matches = no_matches.loc[no_matches['_merge'] != 'both']

In [57]:
no_matches = no_matches[['Student Name', 'Student ID', 'Membership Days', '_merge']]
no_matches.to_csv('data/no_matches.csv', index=False)

### 3B. Look at `Membership = 0`

In [ ]:
merged_filtered['which_df'] = np.where(
     merged_filtered['_merge'] == "left_only", 
    'aspen', 
     np.where(
        merged_filtered['_merge'] == "right_only", 'dashboard', 'both'
     )
)

## 4. Replicate Attendance Report Numbers